In [1]:
import azureml.core 
print(azureml.core.VERSION)

1.20.0


In [1]:
from azureml.core.environment import Environment
Environment(name="vaenv")
myenv = Environment(name="vaenv")

In [2]:
from azureml.core.conda_dependencies import CondaDependencies

dependencies = CondaDependencies()
dependencies.add_pip_package("scikit-learn")
dependencies.add_pip_package("numpy==1.15.4")
myenv.python.conda_dependencies=dependencies

In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name='Toria_workspace',
            subscription_id='784d8740-4c70-4d20-8f45-b2bfecf94029',
            resource_group='udacityprojects',)
exp = Experiment(workspace=ws, name="Toria_workspace")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: Toria_workspace
Azure region: eastus
Subscription id: 784d8740-4c70-4d20-8f45-b2bfecf94029
Resource group: udacityprojects


In [4]:
ws = Workspace.from_config()
experiment_name = 'People_hyperdrive'

experiment=Experiment(ws, experiment_name)

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal,choice
import os
import shutil
from azureml.core import ScriptRunConfig
from azureml.core import Environment


# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.4)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "learning_rate": normal(10, 3),
    "keep_probability": uniform(0.05, 0.1),
    "--C": uniform(0.02, 1),
    "--max_iter": choice(50, 100, 200),

}) 

if "training" not in os.listdir():
    os.mkdir("training")
    print("Path created")
    shutil.copy("traintest2.py","./training")
    print('copied')

#TODO: Create your estimator and hyperdrive config
compute_target = ws.compute_targets['vacompute']
src = ScriptRunConfig(source_directory='.',
                      script='traintest2.py',
                      arguments=["learning_rate", "keep_probability","--C","--max_iter"],
                      compute_target=compute_target,
                      environment = myenv)

hyperdrive_run_config = HyperDriveConfig(estimator=src,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=4)

Path created
copied


In [6]:
from azureml.core import Experiment

run = Experiment(ws,'People_hyperdrive').submit(src)
run.wait_for_completion(show_output=True)

RunId: People_hyperdrive_1611240579_ebe3a95d
Web View: https://ml.azure.com/experiments/People_hyperdrive/runs/People_hyperdrive_1611240579_ebe3a95d?wsid=/subscriptions/784d8740-4c70-4d20-8f45-b2bfecf94029/resourcegroups/udacityprojects/workspaces/toria_workspace

Streaming azureml-logs/55_azureml-execution-tvmps_4f1d66d04d6907de7c68933941e204715d30e2b73fdac290037e0d0395ba4afd_d.txt

2021-01-21T14:50:03Z Starting output-watcher...
2021-01-21T14:50:03Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-01-21T14:50:05Z Executing 'Copy ACR Details file' on 10.0.0.5
2021-01-21T14:50:05Z Copy ACR Details file succeeded on 10.0.0.5. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_1977b0efb94134e883afa1fc00bc0dee
Digest: sha256:2a2087e18514075154ecb8cba883744eae05c28934dfd65174e91ca44e746cd3
Status: Image is up to date for 19ae6d3d824e4474b2a0b61ef254ce10.azurecr.io/azureml/azureml_1977b0efb94134e883afa1fc00bc0dee:latest
19

{'runId': 'People_hyperdrive_1611240579_ebe3a95d',
 'target': 'vacompute',
 'status': 'Completed',
 'startTimeUtc': '2021-01-21T14:50:01.663792Z',
 'endTimeUtc': '2021-01-21T14:51:04.694613Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '97137cf1-cb7b-4bc6-b564-5cc6a8051837',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'traintest2.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['learning_rate', 'keep_probability', '--C', '--max_iter'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'vacompute',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'environment': {'name': 'vaenv',
   'version': 'Autosave_20

In [8]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [12]:
best_model = run.get_metrics()
best_model

{'Regularization Strength:': 1.0,
 'Max iterations:': 1000,
 'Accuracy': 0.9633333333333334}

In [11]:
#TODO: Save the best model
import joblib
joblib.dump(best_model, 'best_remoteml_model.pkl')

['best_remoteml_model.pkl']